<a href="https://colab.research.google.com/github/AndyCatruna/Vision-Summer-School/blob/main/Lab_01_Neural_Networks_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1 - Introduction to PyTorch

The purpose of this lab is to offer a short introduction to the PyTorch library and to help you construct and train a neural network.

You should be familiar with numpy basics. Here is a short [tutorial](https://numpy.org/devdocs/user/quickstart.html) on numpy operations.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### Tensors

PyTorch Tensors are similar to Numpy arrays, but have support for GPU acceleration and gradient computation. A tensor is a generalization of data structures that you are familiar with. For example a vector is a 1D tensor, and a matrix a 2D tensor. Most operations with torch tensors are similar to those of Numpy arrays.

#### Initialization

Creating an empty tensor

In [ ]:
a = torch.tensor([])
print(a)

Creating a tensor with specific size

In [ ]:
b = torch.tensor((3, 4))  # Creates a tensor of size 3x4 filled with uninitialized values
print(b)

Creating a tensor from a list

In [ ]:
c = torch.tensor([[1, 2], [3, 4]])
print(c)

Converting a NumPy array to a tensor

In [ ]:
array_data = np.array([1, 2, 3, 4, 5])
d = torch.from_numpy(array_data)
print(d)

Creating a tensor with random values between 0 and 1 with specific shape

In [ ]:
e = torch.rand(2, 2)
print(e)

#### Shape of tensor

The dimensions of a tensor can be checked with `.shape` attribute or `.size()` function.

In [ ]:
x = torch.rand(5,3,2)

shape = x.shape
print("Shape:", x.shape)

size = x.size()
print("Size:", size)

dim1, dim2, dim3 = x.size()
print("Size:", dim1, dim2, dim3)

#### Operations with tensors

Tensors support most mathematical operations, similar to numpy arrays

Addition

In [ ]:
a = torch.tensor([[1,2], [3,4], [5,6]])
b = torch.rand(3,2)

c = a + b

print("a", a)
print("b", b)
print("c", c)

Elementwise Multiplication

In [ ]:
d = a * b
print(d)

Changing shape of tensor

In [ ]:
e = torch.arange(6)
print("Before", e)
e = e.view(2,3)
print("After", e)

Matrix Multiplication

In [ ]:
f = torch.matmul(a,e)
print(f)

In [ ]:
x = torch.arange(16)
x = x.view(2, 2, 4)
print(x)
print(x.shape)

Indexing

In [ ]:
print(x[0])
print(x[:, 0])
print(x[:,:,0])
print(x[:,:,1:3])

#### Computation Graph

Gradient tracking refers to the ability to automatically compute gradients of a function with respect to its inputs or parameters.

By default, tensors do not keep track of gradients. This can be checked with the `requires_grad` attribute.

In [ ]:
x = torch.tensor([2.0, 3.0])
x.requires_grad

We can change this by setting `requires_grad` to `True`

In [ ]:
x.requires_grad_(True)
print(x.requires_grad)

y = torch.tensor([3.0, 4.0], requires_grad=True)
print(y.requires_grad)

Notice the `grad_fn` attribute. It saves the last function that was performed on that tensor so it can compute its gradient (derivative).

In [ ]:
z = x + y
print(z)

z = x * y
print(z)

z = x ** 2
print(z)

z = x.mean()
print(z)

We can compute gradients using the `backward()` function. We can check the gradients with the `.grad` attribute.

In [ ]:
x = torch.tensor([2.0, 3.0], requires_grad=True)
print(x)

y = x + 2
print(y)

z = y * y * 3
z = z.mean()
print(z)

z.backward() # dz/dx
print("Gradients of z with respect to x: ", x.grad)

#### GPU

For this part you need to select Runtime -> Change Runtime type -> Hardware accelerator: GPU and press save. You will need to rerun all the cells.

Deep learning involves many operations that can be parallelized when working with tensors. Because of this, executing the computation on GPU leads to significantly faster processing.

You can check that GPU acceleration is available with the following command:

In [ ]:
torch.cuda.is_available()

We can specify the device on which all operations are done with the following command:

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

Pushing a tensor on the GPU device is done with the `.to(...)` or `.cuda()` functions. All the operations performed with these tensors will be computed on the GPU.

In [ ]:
x = torch.tensor([2.0, 3.0])
x = x.to(device)
print(x)

y = torch.tensor([5.0, 1.0]).to(device)
print(y)

z = x + y
print(z)

# Part 2 - Training Neural Networks

#### Gradient Descent

Gradient descent is an optimization algorithm used to iteratively adjust model parameters based on the computed gradients, moving in the direction of steepest descent to find the optimal values that minimize the loss function.

<img src="https://sebastianraschka.com/images/faq/gradient-optimization/ball.png" width=400px>

Gradient descent can be described by the following formula:

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20200611183120/1406-7.png">

where $\theta$ are the weights (trainable parameters) of the model, $\alpha$ is the learning rate and $J(\theta)$ is the cost function (loss).

The weights are updated in the opposite direction of the derivative (gradient) of the cost function. This basically changes the parameters so that the loss decreases.

Remember that PyTorch can automatically track the gradients for us, which means we do not have to compute derivatives manually.

The steps of training a neural network are the following:

1. Initialize parameters.
2. Compute loss.
3. Calculate gradients of loss with respect to the parameters.
4. Update parameters by moving in the opposite direction of the gradients.
5. Repeat steps 2-4 for multiple epochs.
6. Stop when a stopping condition is met (reaching a desired loss or number of epochs).



#### Training a Linear Model

We will train a simple linear model to learn the simple function $f(x) = 2*x$ using gradient descent in order to get familiar with the training pipeline.

The model will have a single parameter $w \in R$ and the output of the model will be $pred(x) = w * x$.

The model should learn that $w = 2$ if we show it enough examples.

We create a few examples that the model will see. $x$ is the input and $y$ is the ground truth.

In [ ]:
# Dummy dataset
x = torch.tensor([1,2,3,4,5], dtype=torch.float32)
y = torch.tensor([2,4,6,8,10], dtype=torch.float32)

We define the hyperparameters of training the model. These are chosen by the developer and can be changed to obtain better results

In [ ]:
# Hyperparameters
learning_rate = 0.01
epochs = 100

We initialize the weights of the model ($w=0$) and define the forward function (how the model obtains the output based on the input).

In [ ]:
# 1. Initialize parameters
weights = torch.tensor(0.0, requires_grad=True)

# Define forward function for linear model
def forward(x):
  return weights * x

We define the loss function $J$. This should tell us how close the model's predictions are to the ground truth. We choose mean absolute error as it is intuitive, but there are other loss functions. Most of them are already available in pytorch and we don't need to manually define them. You can check them [here](https://pytorch.org/docs/stable/nn.html#loss-functions).

In [ ]:
# Define the custom loss (mean absolute error)
def custom_loss(pred, ground_truth):
  return torch.abs(pred - ground_truth).mean()

We want to see if the model will be able to predict the correct answer even for unseen samples. For example, $6$ is not in the samples in $x$.

In [ ]:
with torch.no_grad():
  print(f"Prediction before training: f(6) = {forward(6)}")

We perform the training loop:
* we obtain the model's prediction
* we see how good the predictions are based on the loss
* we update the model's weights in the direction that minimizes the loss.

In [ ]:
for epoch in range(epochs):

  # Obtain prediction
  pred = forward(x)

  # 2. Compute loss
  loss = custom_loss(pred, y)

  # 3. Compute gradients
  loss.backward()

  # Gradient of loss with respect to weights
  dw = weights.grad

  # 4. Update parameters (without grad so it does not mess with computation - this will be removed when working with torch optimizers)
  with torch.no_grad():
    weights -= (learning_rate * dw)

  # Set gradient to 0 for next computation
  weights.grad.zero_()

  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, weights = {weights}, loss = {loss}")

print(f"Prediction after training: f(6) = {forward(6)}")


Notice the final weight $w$ is not exactly equal to 2 but is very close.

This can be improved with a smaller learning rate and maybe more training epochs.

However, in machine learning we don't expect perfect solutions.

**Exercise 1** - change the hyperparameters and observe how the progress of weights changes.

Luckily we do not have to manually perform the updates of the model.

We can do this by defining the `optimizer = optim.SGD(...)` which does the gradient descent updates for us.

Also, we do not need to manually define the weights. We can use `nn.Module` class and `nn.Linear(...)` for this.

We construct a simple linear model by inheriting from `nn.Module` class.

All neural networks you will build will inherit from this class. It's good to get familiar with it. You can check more details about it [here](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).

In [ ]:
class LinearModel(nn.Module):
  def __init__(self):
    super(LinearModel, self).__init__()
    # One input neuron and one output neuron
    self.weights = nn.Linear(in_features=1, out_features=1, bias=False)

  def forward(self, x):
    x = self.weights(x)

    return x

We initialize the model and define the hyperparameters

In [ ]:
# 1. Initialize parameters
model = LinearModel()
print(model)

# Hyperparameters
learning_rate = 0.01
epochs = 100

We initialize the optimizer. We will use `optim.SGD` which stands for Stochastic Gradient Descent. In the PyTorch library there are optimizers that may work better than Stochastic Gradient Descent (for example `optim.Adam` which is very commonly used). You can check them [here](https://pytorch.org/docs/stable/optim.html)

In [ ]:
# This will perform Gradient Descent for us
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

The new model expects the samples to come in batches. We will add an extra dimension to our $x$ and $y$ tensors.

In [ ]:
print(x.shape, y.shape)
# Add extra dimension for batch:
x = x.unsqueeze(1)
y = y.unsqueeze(1)
print(x.shape, y.shape)

# Define testing sample
testing_sample = torch.tensor([[6.0]])

Prediction on unseen sample before training

In [ ]:
with torch.no_grad():
  print(f"Prediction before training: f(6) = {model(testing_sample)}")

Now the training loop looks like this:

In [ ]:
for epoch in range(epochs):

  # Obtain prediction

  pred = model(x)

  # 2. Compute loss
  loss = custom_loss(pred, y)

  # 3. Compute gradients
  loss.backward()

  # 4. Update parameters
  optimizer.step()

  # Set gradient to 0 for next computation
  optimizer.zero_grad()

  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, loss = {loss}")

with torch.no_grad():
  print(f"Prediction after training: f(6) = {model(testing_sample)}")


#### Neural Network on Circle Dataset

We will build the following Neural Network with 2 input neurons, 3 neurons in the hidden layer and two output neurons:

<img src="https://www.researchgate.net/publication/289479445/figure/fig1/AS:614019022991383@1523404951564/Example-for-an-artificial-neural-network-with-two-input-neurons-two-hidden-neurons-and.png" width=300px>

Each neuron in the neural network is just a linear model + an activation function (which adds nonlinearity).

All the neurons in a previous layer are connected to all the layers in the current layer.

This means that the output of a neuron will be computed based on all the outputs of the neurons in the previous layer.

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    # This defines the connections from input layer to hidden layer - 2 neruons to 3 neurons
    self.fc1 = nn.Linear(in_features=2, out_features=3)

    # This defines the connections from hidden layer to output layer - 3 neruons to 2 neurons
    self.fc2 = nn.Linear(in_features=3, out_features=2)

    # This is a commonly used nonlinearity
    self.activation = nn.ReLU()

    # Activation functions which are more recent:
    # self.activation = nn.GELU()
    # self.activation = nn.Mish()
    # self.activation = nn.ELU()

    # Other activation functions (only used in specific cases):
    # self.activation = nn.Tanh()
    # self.activation = nn.Sigmoid()

  def forward(self, x):
    # We pass through first layer
    x = self.fc1(x)

    # We add nonlinearity
    x = self.activation(x)

    # We pass through output layer
    x = self.fc2(x)

    return x

We will train it to classify samples consisting of points inside or outside a circle. We will have 2 classes: $inside=0$ and $outside=1$.



In [ ]:
# This is the code that generates the samples - you can ignore it

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
torch.manual_seed(42)

# Define the circle parameters
radius = 5
center = (0, 0)

# Generate the dataset
num_samples = 2000

# Generate random x, y points within a square
x = torch.rand(num_samples) * 2 * radius - radius
y = torch.rand(num_samples) * 2 * radius - radius

# Calculate the Euclidean distance from each point to the center
distances = torch.sqrt((x - center[0]) ** 2 + (y - center[1]) ** 2)

# Assign labels based on whether points are inside or outside the circle
labels = torch.where(distances <= radius, torch.zeros_like(distances), torch.ones_like(distances))

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
labels_train, labels_test = train_test_split(labels, test_size=0.2, random_state=42)

# Plot the training and testing datasets
plt.scatter(x_train[labels_train == 0], y_train[labels_train == 0], color='blue', label='Inside Circle (Train)')
plt.scatter(x_train[labels_train == 1], y_train[labels_train == 1], color='red', label='Outside Circle (Train)')
plt.scatter(x_test[labels_test == 0], y_test[labels_test == 0], color='cyan', label='Inside Circle (Test)')
plt.scatter(x_test[labels_test == 1], y_test[labels_test == 1], color='orange', label='Outside Circle (Test)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Dataset')
plt.legend()
plt.show()

Create Dataset class which stores samples and their labels.

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CircleDataset(Dataset):
    def __init__(self, x, y, labels):
        self.x = x
        self.y = y
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            'coords': torch.tensor([self.x[idx], self.y[idx]]),
            'label': self.labels[idx].long()
        }

        return sample

The Dataset class in PyTorch represents a collection of data samples and labels. It is needed for the DataLoader class.

The DataLoader class efficiently loads and manages the data during training or inference by providing features like batching and shuffling. These classes are commonly used in PyTorch. You can read more about them [here](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

In [ ]:
train_dataset = CircleDataset(x_train, y_train, labels_train)
test_dataset = CircleDataset(x_test, y_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

We will define an evaluation function which calculates the accuracy on the test set. This function will be used in the training loop in order to keep track of the evolution of the model on unseen data.

We compute the accuracy manually for educational purposes. However, this is not necessary. There are already functions for computing evaluation metrics in the sklearn library. You can check them [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics). See [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score) and [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score).

In [ ]:
def evaluate(model):
  # Put model in evaluation mode
  model.eval()

  # For keeping track of number of correct predictions and total predictions
  correct = 0
  total = 0

  for data in test_loader:
      coords = data['coords'].to(device)
      ground_truth = data['label'].to(device)

      # We do not need to keep track of gradients while testing
      with torch.no_grad():
        pred = model(coords)

      # From list of predicted scores we get the class with the highest score
      _, predicted = torch.max(pred, 1)

      # We count all the predictions which match the ground truth to get number of correct predictions
      correct += (predicted == ground_truth).sum().item()
      total += coords.shape[0]

  accuracy = np.round(100 * correct / total, 2)

  print(f"Test Accuracy: {accuracy}")

The training loop will look very similar to the previous example.

However, now we also iterate through the data loader which returns batches of random samples and their labels.

In [ ]:
# Cross Entropy Loss - commonly used for classification
criterion = nn.CrossEntropyLoss()

# Hyperparameters
learning_rate = 0.0005
epochs = 10

# 1. Initialize parameters
model = Net()

# Push model on device
model.to(device)
print(model)

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
  epoch_loss = 0
  for data in train_loader:
    # Put model in training mode
    model.train()

    coords = data['coords'].to(device)
    ground_truth = data['label'].to(device)
    pred = model(coords)

    # 2. Compute loss
    loss = criterion(pred, ground_truth)

    # 3. Compute gradients
    loss.backward()

    # 4. Update parameters
    optimizer.step()

    # Set gradient to 0 for next computation
    optimizer.zero_grad()

    epoch_loss += loss

  print(f"Epoch: {epoch}, Training loss = {epoch_loss}")
  evaluate(model)


**Exercise 2** - Play with the hyperparameters (learning rate, number of epochs) and with the structure of the model (add more neurons in the hidden layer, add more layers) and see if you can improve the results. Maybe also change the optimizer.

Try to apply only one change at a time and test it to see what improves the results and what decreases performance.

#### Neural Network for classification on the Iris Dataset

**Exercise 3** - Train a Neural Network for classification on the iris dataset.

The iris dataset consists of measurements of sepal and petal lengths and widths for three different species of iris flowers: setosa, versicolor, and virginica.

Your input to the model will be all these measurements (features) and you will have 3 flower classes.

This code imports the dataset and creates the dataloaders that you need

In [ ]:
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()
data = iris.data
labels = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

# Split the dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

Plotting the dataset

In [ ]:
# Create plots for different feature combinations
feature_combinations = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
num_plots = len(feature_combinations)
num_rows = (num_plots + 1) // 3  # Calculate the number of rows required for the subplot grid

# Create the subplot grid
fig, axs = plt.subplots(num_rows, 3, figsize=(10, num_rows * 3))

# Plotting the dataset for each feature combination
for i, (feat_idx1, feat_idx2) in enumerate(feature_combinations):
    row = i // 3  # Determine the row index in the subplot grid
    col = i % 3   # Determine the column index in the subplot grid
    axs[row, col].scatter(data[:, feat_idx1], data[:, feat_idx2], c=labels, cmap='viridis')
    axs[row, col].set_xlabel(feature_names[feat_idx1])
    axs[row, col].set_ylabel(feature_names[feat_idx2])
    axs[row, col].set_title(f"{feature_names[feat_idx1]} vs {feature_names[feat_idx2]}")

plt.tight_layout()
plt.show()


We create the Dataset class

In [ ]:
class IrisDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            'features': torch.tensor(self.features[idx], dtype=torch.float32),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }
        return sample

We create the dataloaders for training set and testing set

In [ ]:
# Create train and test datasets
iris_train_dataset = IrisDataset(x_train, y_train)
iris_test_dataset = IrisDataset(x_test, y_test)

# Create train and test data loaders
iris_train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
iris_test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Your job is to construct the neural network and write the training and evaluation

In [ ]:
class MyNet(nn.Module):
  def __init__(self):
    super(MyNet, self).__init__()
    # TODO - Write the layers of the network

  def forward(self, x):
    # TODO - Write the forward function - pass through each layer - do not forget about activation functions
    pass

Write the training loop and the evaluation:

* Define the hyperparameters
* Define the loss function (criterion) - You can use already existing ones.
* Define the optimizer
* Initialize the model and train it
* Evaluate the model's accuracy

You can take inspiration from the previous example.

In [ ]:
# TODO - Write the training loop and evaluation code on the test set